# Data cleaning and merging

In [1]:
import pandas as pd
import us
import numpy as np
from tqdm import tqdm

In [2]:
# Data from Census API calls
cen = pd.read_csv('/Users/alexandrabruno/Documents/flatiron/data/census_final.csv')

/Users/alexandrabruno/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


**GIVING OUR DATAFRAME A HEADER**

In [3]:
# Deleting Unnamed: 0 column
del cen['Unnamed: 0']

# Creating a list of our column names 
new_header = [
    'PWGTP',
    'AGEP',
    'FER',
    'SEX',
    'RACBLK',
    'RACWHT',
    'RACSOR',
    'HISP',
    'SCHL',
    'ST',
    'YEAR'
]

# Setting our column names equal to our list 
cen.columns = new_header

In [4]:
# Get indexes where name column has value PWGTP
indexNames = cen[cen['PWGTP'] == 'PWGTP'].index
 
# Delete these row indexes from dataFrame
cen.drop(indexNames, inplace=True)

**CHANGING DATA TYPES**

In [5]:
# Change columns into the data types we need so we can use them for creating our features
cen.FER = cen.FER.astype(int)
cen.HISP = cen.HISP.astype(int)
cen.RACWHT = cen.RACWHT.astype(int)
cen.RACBLK = cen.RACBLK.astype(int)
cen.AGEP = cen.AGEP.astype(int)
cen.ST = cen.ST.astype(str)
cen.SCHL = cen.SCHL.astype(int)

**DROPPING ROWS WE DON'T NEED**

We aren't interesting in all of the data within this data frame so we are going to drop the irrelevant rows

In [6]:
# Get indexes where name column has value 0
indexNames = cen[cen['FER'] == 0].index
 
# Delete these row indexes from dataFrame
cen.drop(indexNames, inplace=True)

# Get indexes where name column has value 8
indexNames = cen[cen['FER'] == 8].index
 
# Delete these row indexes from dataFrame
cen.drop(indexNames, inplace=True)

# Get indexes where name column has value > 46
indexNames = cen[cen['AGEP'] >= 46].index
 
# Delete these row indexes from dataFrame
cen.drop(indexNames, inplace=True)

# Get indexes where name column has value < 14 
indexNames = cen[cen['AGEP'] <= 14].index
 
# Delete these row indexes from dataFrame
cen.drop(indexNames, inplace=True)

**CHANGING THE ROW VALUES IN OUR ST COLUMN**

The census data gave the state values their fip codes and we want to change them to their state names

In [7]:
# Creating a list of the fip codes
fip_codes = ['1','2','4','5','6','8','9','10','11','12','13','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']

In [8]:
# Creating a list of the states
states = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado',
         'Connecticut','Delaware','District of Columbia','Florida','Georgia','Hawaii','Idaho','Illinois',
          'Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland',
         'Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana',
         'Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York',
         'North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island',
          'South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington',
          'West Virginia','Wisconsin','Wyoming']

In [9]:
# Zipping our list into a dicitionary so we can replace the old values with the new ones 
changes = dict(zip(fip_codes,states))

In [10]:
# Replacing the values with a lambda function 
cen['STATE'] = cen['ST'].apply(lambda x: changes[x])

**CHANGING OUR SCHL VARIABLE**

The other datasets that we plan on merging with this one have ages in categorical bins. We are going to use pd.cut to group our ages within specified ranges to match our other datasets

In [13]:
cen['EDUC'] = pd.cut(cen.SCHL, [1,16,18,21,42], labels=['less hs','hs','som col','col or more'],right=False)

**CHANGING OUR AGEP VARIABLE**

The other datasets that we plan on merging with this one have ages in categorical bins. We are going to use pd.cut to group our ages within specified ranges to match our other datasets

In [16]:
cen['AGE'] = pd.cut(cen.AGEP, [15,20,25,30,35,40,45], labels=['15 to 19', '20 to 24', '25 to 29','30 to 34', '35 to 39','40 to 45'],right=False)

In [ ]:
# Making our education variables dummies
cen = pd.get_dummies(cen, columns=['EDUC'])

In [ ]:
cen = cen.rename(columns={'EDUC_col or more': 'COLPLUS', 'EDUC_hs':'HS', 'EDUC_less hs': 'LESSHS','EDUC_som col':'SOMCOL'})

**TURNING OUR HISPAN VARIABLE INTO DUMMY**

We are just interested in the percent of fertile respondents that are Hispanic, for that reason we are not going to be examining their Hispanic origin.

In [17]:
cen['HISPAN'] = np.where(cen['HISP']>1, 1, 0)

In [3]:
# Saving this data so we do not have to rerun the processes above 
cen.to_csv('/Users/alexandrabruno/Documents/flatiron/data/for_agg.csv')

NameError: name 'cen' is not defined

In [2]:
cen = pd.read_csv('/Users/alexandrabruno/Documents/flatiron/data/for_agg.csv')

In [10]:
states = list(cen['STATE'].unique())
years = list(cen['YEAR'].unique())
ages = list(cen['AGE'].unique())

**GETTING THE PERCENTAGE OF RESPONDENTS THAT ARE FERTILE FOR EACH RACIAL GROUP**

We are looping through the lists below to calculate the number of respondents that are fall in each age group, in each each, and in each state and calculating the amount of them that are white, black, hispanic, or other. 

In [4]:
lst = []
for state in tqdm(states):
    for year in years:
            for age in ages:
                try:
                    perc_blk = cen.loc[(cen['YEAR']==year) & (cen['RACBLK']== 1) & (cen['STATE']== state) & (cen['AGE'] == age)]['FER'].value_counts(normalize = True)[1]
                    cen.loc[(cen['YEAR']==year) & (cen['RACBLK']== 1) & (cen['STATE']== state) & (cen['AGE'] == age), 'PRC_BLK'] = perc_blk
                except:
                    lst.append([state,year,age]) 
                    continue 

100%|██████████| 51/51 [2:42:37<00:00, 191.33s/it]  


In [5]:
cen.to_csv('/Users/alexandrabruno/Documents/flatiron/data/agg1.csv')

In [6]:
lst = []
for state in tqdm(states):
    for year in years:
            for age in ages:
                try:
                    perc_wht = cen.loc[(cen['YEAR']==year) & (cen['RACWHT']== 1) & (cen['STATE']== state) & (cen['AGE'] == age)]['FER'].value_counts(normalize = True)[1]
                    cen.loc[(cen['YEAR']==year) & (cen['RACWHT']== 1) & (cen['STATE']== state) & (cen['AGE'] == age), 'PRC_WHT'] = perc_wht
                except:
                    lst.append([state,year,age]) 
                    continue 

100%|██████████| 51/51 [2:58:38<00:00, 210.16s/it]  


In [7]:
cen.to_csv('/Users/alexandrabruno/Documents/flatiron/data/agg2.csv')

In [8]:
lst = []
for state in tqdm(states):
    for year in years:
            for age in ages:
                try:
                    perc_hisp = cen.loc[(cen['YEAR']==year) & (cen['HISPAN']== 1) & (cen['STATE']== state) & (cen['AGE'] == age)]['FER'].value_counts(normalize = True)[1]
                    cen.loc[(cen['YEAR']==year) & (cen['HISPAN']== 1) & (cen['STATE']== state) & (cen['AGE'] == age), 'PRC_HISP'] = perc_hisp
                except:
                    lst.append([state,year,age]) 
                    continue 

100%|██████████| 51/51 [2:45:31<00:00, 194.74s/it]  


In [9]:
cen.to_csv('/Users/alexandrabruno/Documents/flatiron/data/agg3.csv')

In [11]:
lst = []
for state in tqdm(states):
    for year in years:
            for age in ages:
                try:
                    perc_other = cen.loc[(cen['YEAR']==year) & (cen['RACSOR']== 1) & (cen['STATE']== state) & (cen['AGE'] == age)]['FER'].value_counts(normalize = True)[1]
                    cen.loc[(cen['YEAR']==year) & (cen['RACSOR']== 1) & (cen['STATE']== state) & (cen['AGE'] == age), 'PRC_OTHER'] = perc_other
                except:
                        lst.append([state,year,age]) 
                        continue 

100%|██████████| 51/51 [2:39:49<00:00, 188.03s/it]  


In [12]:
cen.to_csv('/Users/alexandrabruno/Documents/flatiron/data/agg4.csv')

**GETTING THE PERCENTAGE OF RESPONDENTS THAT ARE FERTILE FOR EACH EDUCATIONAL GROUP**

We are looping through the lists below to calculate the number of respondents that are fall in each age group, in each each, and in each state and calculating the amount of them that have less than a hs degree, a hs degree, some college education, a college degree, and more than a college degree. 

In [11]:
lst = []
for state in tqdm(states):
    for year in years:
            for age in ages:
                try:
                    lesshs = cen.loc[(cen['YEAR']==year) & (cen['LESSHS']== 1) & (cen['STATE']== state) & (cen['AGE'] == age)]['FER'].value_counts(normalize = True)[1]
                    cen.loc[(cen['YEAR']==year) & (cen['LESSHS']== 1) & (cen['STATE']== state) & (cen['AGE'] == age), 'LESS_HS'] = lesshs
                except:
                    lst.append([state,year,age]) 
                    continue 

100%|██████████| 51/51 [3:04:22<00:00, 216.92s/it]  


In [12]:
cen.to_csv('/Users/alexandrabruno/Documents/flatiron/data/agg5.csv')

In [13]:
lst = []
for state in tqdm(states):
    for year in years:
            for age in ages:
                try:
                    hs = cen.loc[(cen['YEAR']==year) & (cen['HS']==1) & (cen['STATE']== state) & (cen['AGE'] == age)]['FER'].value_counts(normalize = True)[1]
                    cen.loc[(cen['YEAR']==year) & (cen['HS']==1) & (cen['STATE']== state) & (cen['AGE'] == age), '_HS'] = hs
                except:
                    lst.append([state,year,age]) 
                    continue 

100%|██████████| 51/51 [2:36:46<00:00, 184.43s/it]  


In [14]:
cen.to_csv('/Users/alexandrabruno/Documents/flatiron/data/agg6.csv')

In [15]:
lst = []
for state in tqdm(states):
    for year in years:
            for age in ages:
                try:
                    somecol = cen.loc[(cen['YEAR']==year) & (cen['SOMCOL']==1) & (cen['STATE']== state) & (cen['AGE'] == age)]['FER'].value_counts(normalize = True)[1]
                    cen.loc[(cen['YEAR']==year) & (cen['SOMCOL']==1) & (cen['STATE']== state) & (cen['AGE'] == age), 'SOM_COL'] = somecol
                except:
                    lst.append([state,year,age]) 
                    continue 

100%|██████████| 51/51 [2:28:43<00:00, 174.97s/it]  


In [16]:
cen.to_csv('/Users/alexandrabruno/Documents/flatiron/data/agg7.csv')

In [17]:
lst = []
for state in tqdm(states):
    for year in years:
            for age in ages:
                try:
                    colplus = cen.loc[(cen['YEAR']==year) & (cen['COLPLUS']== 1) & (cen['STATE']== state) & (cen['AGE'] == age)]['FER'].value_counts(normalize = True)[1]
                    cen.loc[(cen['YEAR']==year) & (cen['COLPLUS']== 1) & (cen['STATE']== state) & (cen['AGE'] == age), 'COL_PLUS'] = colplus
                except:
                    lst.append([state,year,age]) 
                    continue 

100%|██████████| 51/51 [2:20:50<00:00, 165.69s/it]  


In [18]:
cen.to_csv('/Users/alexandrabruno/Documents/flatiron/data/agg8.csv')

**GETTING THE PERCENTAGE OF RESPONDENTS THAT ARE FERTILE FOR EACH EDUCATIONAL GROUP**

We are looping through the lists below to calculate the number of respondents that are fall in each age group, in each each, and in each state and calculating the amount of them that have less than a hs degree, a hs degree, some college education, a college degree, and more than a college degree. 

# Merging

In [53]:
chl = pd.read_csv('/Users/alexandrabruno/Documents/flatiron/data/Chlamydia.csv')

In [55]:
chl

,Unnamed: 0,Indicator,Year,Geography,Race/Ethnicity,Age_group,Cases,"Rate_per_100,000"
0,0,Chlamydia,2018,Alabama,White,45-54,26,11.8
1,1,Chlamydia,2018,Alabama,White,40-44,28,29.7
2,2,Chlamydia,2018,Alabama,White,35-39,66,69.5
3,3,Chlamydia,2018,Alabama,White,30-34,185,195.9
4,4,Chlamydia,2018,Alabama,White,25-29,437,422.6
...,...,...,...,...,...,...,...,...
25671,35814,Chlamydia,2008,Wyoming,American Indian/Alaska Native,35-39,0,0.0
25672,35815,Chlamydia,2008,Wyoming,American Indian/Alaska Native,30-34,3,849.9
25673,35816,Chlamydia,2008,Wyoming,American Indian/Alaska Native,25-29,6,1336.3
25674,35817,Chlamydia,2008,Wyoming,American Indian/Alaska Native,20-24,14,2772.3


In [57]:
chl['Year'].unique()

array([2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008,
       2007])

In [60]:
chl['Age_group'].value_counts()

25-29    3668
15-19    3668
40-44    3668
20-24    3668
35-39    3668
30-34    3668
45-54    3668
Name: Age_group, dtype: int64

In [61]:
gbd['age'].value_counts()

25 to 29    15300
45 to 49    15300
15 to 19    15300
40 to 44    15300
35 to 39    15300
30 to 34    15300
20 to 24    15300
Name: age, dtype: int64

In [63]:
gbd.columns

Index(['measure', 'location', 'sex', 'age', 'cause', 'metric', 'year', 'val',
       'upper', 'lower'],
      dtype='object')

In [62]:
chl.columns

Index(['Unnamed: 0', 'Indicator', 'Year', 'Geography', 'Race/Ethnicity',
       'Age_group', 'Cases', 'Rate_per_100,000'],
      dtype='object')

In [64]:
chl = chl.rename(columns={'Geography': 'STATE', 'Age_group': 'AGE', 'Rate_per_100,000':'CHL_RTE'})

In [66]:
chl = chl.rename(columns={'Year': 'YEAR'})

In [ ]:
gbd = gbd.rename(columns={'year': 'YEAR','age':'AGE',})